### Thoughts 

We set up a looper function - that runs through each labelled 30 second segment 
For (30 sec segment call X(x,y) where x is raw data and y is labe) :
    - feature extraction function that applies to it returns some vector/scalar of values (doubles) then appends on 
    This means that you can mix and match which features you want to be put into the loop 

At end of the loop you know have a vector of extracted features in the order that you placed the feature functions in with a label this is then saved to the feature extracted data file. 

In [1]:
import json
import logging
import os
import pdb
from pathlib import Path
from scipy import signal 

import mne
import numpy as np
import pandas as pd

In [2]:
# Load the clean data you saved in the previous notebook.
cleaned_data_file = '/Users/katieoreilly/Desktop/unsw/SCIF_2001_actual2.nosync/SCIF_2001/EEG_data/cleaned_data/sub-001_cleaned-epo.fif'
epochs = mne.read_epochs(cleaned_data_file, preload=True)
raw_cleaned_df = epochs.to_data_frame()
def select_channels(cleaned_df) : 
    #write code to delete the channels from the EEG that you dont want to work with here 
    #and have the function return the new data frame
    return 

cleaned_df = select_channels(raw_cleaned_df) 

Reading /Users/katieoreilly/Desktop/unsw/SCIF_2001_actual2.nosync/SCIF_2001/EEG_data/cleaned_data/sub-001_cleaned-epo.fif ...
    Found the data of interest:
        t =       0.00 ...   30000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1435 matching events found
No baseline correction applied
0 projection items activated


In [5]:



def extract_hjorth_parameters(epoch):
    """
    Calculates Hjorth parameters (Activity, Mobility, Complexity) for each channel.
    
    Returns:
        tuple: Three numpy arrays (activity, mobility, complexity), 
               each of shape (n_channels,).
    """
    # Calculate the first and second derivatives
    d1 = np.diff(epoch, axis=1)
    d2 = np.diff(d1, axis=1)
    
    # Variance of the signal and its derivatives
    var_0 = np.var(epoch, axis=1)
    var_d1 = np.var(d1, axis=1)
    var_d2 = np.var(d2, axis=1)
    
    # Activity is just the variance of the signal
    activity = var_0
    
    # Mobility is the ratio of the std of the derivative to the std of the signal
    mobility = np.sqrt(var_d1 / var_0)
    
    # Complexity is the ratio of the std of the second derivative to the std of the first
    complexity = np.sqrt(var_d2 / var_d1) / mobility
    
    return activity, mobility, complexity
    

def extract_band_power(epoch, sf):
    """
    Extracts the relative power in different frequency bands for each channel.

    Args:
        epoch (np.ndarray): The EEG data for a single epoch (n_channels,n_samples).
        sf (int): The sampling frequency of the EEG data.

    Returns:
        dict: A dictionary where keys are band names (e.g., 'delta') and
              values are numpy arrays of the power for each channel.
    """
    bands = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 12),
        'sigma': (12, 15),
        'beta': (15, 30)
    }
    
    # Use Welch's method to get the power spectral density
    # nperseg is the length of each segment. A value between 2-4 seconds is common.
    win = 2 * sf # 2-second window
    freqs, psd = signal.welch(epoch, sf, nperseg=win)
    
    # Calculate absolute power in each band
    band_powers = {}
    for band, (low, high) in bands.items():
        # Find the indices of the frequencies that fall within the band
        idx_band = np.logical_and(freqs >= low, freqs <= high)
        # Integrate the PSD over the band using Simpson's rule for better accuracy
        power = simps(psd[:, idx_band], freqs[idx_band], axis=-1)
        band_powers[band] = power
        
    # Optional: Calculate total power to compute relative power
    # total_power = simps(psd, freqs, axis=-1)
    # relative_powers = {band: power / total_power for band, power in band_powers.items()}
    
    return band_powers # Returning absolute power is usually sufficient for trees

# Want to generate new table based off old data fram 
# Should have functions that edit old data fram to remove channels that we dont want 
# Columns: Y with the label, features each function should generate a new column"





def get_features_for_epoch(epoch, sf):
    """
    Generates a complete feature vector for a single epoch.
    """
    # 1. Frequency-domain features
    band_powers = extract_band_power(epoch, sf)
    delta_p = band_powers['delta']
    theta_p = band_powers['theta']
    alpha_p = band_powers['alpha']
    sigma_p = band_powers['sigma']
    beta_p = band_powers['beta']
    
    # 2. Time-domain statistical features
    var_vals = extract_variance(epoch)
    skew_vals = extract_skewness(epoch)
    kurt_vals = extract_kurtosis(epoch)
    
    # 3. Hjorth parameters
    _, mobility_vals, complexity_vals = extract_hjorth_parameters(epoch) # Activity is same as variance

    # Concatenate all features into a single 1D array
    # The order is [delta_ch1, delta_ch2, ..., theta_ch1, theta_ch2, ..., etc.]
    feature_vector = np.concatenate([
        delta_p, theta_p, alpha_p, sigma_p, beta_p,
        var_vals, skew_vals, kurt_vals,
        mobility_vals, complexity_vals
    ], axis=0)
    
    return feature_vector

### Function layout 
Each function should take in the pre processed EEG data, a vector representing where to store the features, append new features onto the end of the vector. 
The function will return the vector. 

In [ ]:
#Function for Extracting Hjforth parameters 

#Function for extracting spectral power 

#Function for extracting signal kurtosis 

#Function for extracting spectral entropy 

#Function for Extracting Band power rations 

In [ ]:
#Feature engineering and extraction 


#save extracted features as a vector with labelling. 